In [1]:
import os
import math
import numpy as np
import pandas as pd
from datetime import datetime

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

# -----------------------------
# Config
# -----------------------------
CSV_FILE = "/content/XAUUSDh1.csv"   # ensure this file exists in your working directory
DATE_COL = "Date"
SEQ_LEN = 64
VAL_RATIO = 0.15
TEST_RATIO = 0.15
BATCH_SIZE = 256
EPOCHS = 100
LR = 1e-3
NUM_HEADS = 4
KEY_DIM = 64
FF_DIM = 128
NUM_ENCODER_BLOCKS = 2
DROPOUT = 0.1
SEED = 42

tf.random.set_seed(SEED)
np.random.seed(SEED)

# -----------------------------
# Utilities: Indicators & Feature Engineering
# -----------------------------
def ema(series: pd.Series, span: int) -> pd.Series:
    return series.ewm(span=span, adjust=False).mean()

def rsi(series: pd.Series, period: int = 14) -> pd.Series:
    delta = series.diff()
    gain = np.where(delta > 0, delta, 0.0)
    loss = np.where(delta < 0, -delta, 0.0)
    gain = pd.Series(gain, index=series.index)
    loss = pd.Series(loss, index=series.index)
    avg_gain = gain.ewm(alpha=1/period, min_periods=period, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/period, min_periods=period, adjust=False).mean()
    rs = avg_gain / (avg_loss + 1e-12)
    rsi = 100 - (100 / (1 + rs))
    return rsi

def true_range(high: pd.Series, low: pd.Series, close: pd.Series) -> pd.Series:
    prev_close = close.shift(1)
    tr1 = (high - low).abs()
    tr2 = (high - prev_close).abs()
    tr3 = (low - prev_close).abs()
    return pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)

def atr(high: pd.Series, low: pd.Series, close: pd.Series, period: int = 14) -> pd.Series:
    tr = true_range(high, low, close)
    return tr.ewm(alpha=1/period, adjust=False).mean()

def macd(close: pd.Series, fast: int = 12, slow: int = 26, signal: int = 9):
    ema_fast = ema(close, fast)
    ema_slow = ema(close, slow)
    macd_line = ema_fast - ema_slow
    signal_line = ema(macd_line, signal)
    hist = macd_line - signal_line
    return macd_line, signal_line, hist

def bollinger(close: pd.Series, period: int = 20, num_std: float = 2.0):
    ma = close.rolling(window=period, min_periods=period).mean()
    sd = close.rolling(window=period, min_periods=period).std()
    upper = ma + num_std * sd
    lower = ma - num_std * sd
    width = upper - lower
    pctb = (close - lower) / (upper - lower + 1e-12)
    return ma, upper, lower, width, pctb

# -----------------------------
# Load & Prepare Data
# -----------------------------
df = pd.read_csv(CSV_FILE)
# Basic validation
required_cols = {"open", "high", "low", "close", "tick_volume", DATE_COL}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Missing columns in CSV: {missing}")

# Ensure time order
df[DATE_COL] = pd.to_datetime(df[DATE_COL])
df = df.sort_values(DATE_COL).reset_index(drop=True)

# -----------------------------
# Feature Engineering
# -----------------------------
open_ = df["open"].astype(float)
high = df["high"].astype(float)
low = df["low"].astype(float)
close = df["close"].astype(float)
vol = df["tick_volume"].astype(float)

# Core features
df_feat = pd.DataFrame(index=df.index)
df_feat["close"] = close
df_feat["return"] = close.pct_change().fillna(0.0)
df_feat["price_change"] = close.diff().fillna(0.0)
df_feat["hl_spread"] = (high - low)
df_feat["oc_spread"] = (open_ - close)
df_feat["vol_chg"] = vol.diff().fillna(0.0)
df_feat["log_vol"] = np.log1p(vol)

# Volatility
df_feat["ret_vol_24"] = df_feat["return"].rolling(window=24, min_periods=24).std().fillna(method="bfill")
df_feat["ret_vol_72"] = df_feat["return"].rolling(window=72, min_periods=72).std().fillna(method="bfill")

# ATR
df_feat["atr_14"] = atr(high, low, close, period=14).fillna(method="bfill")

# RSI
df_feat["rsi_14"] = rsi(close, period=14).fillna(method="bfill")

# MACD
macd_line, signal_line, hist = macd(close, fast=12, slow=26, signal=9)
df_feat["macd"] = macd_line.fillna(method="bfill")
df_feat["macd_signal"] = signal_line.fillna(method="bfill")
df_feat["macd_hist"] = hist.fillna(method="bfill")

# EMAs
df_feat["ema_20"] = ema(close, 20).fillna(method="bfill")
df_feat["ema_50"] = ema(close, 50).fillna(method="bfill")
df_feat["ema_ratio_20_50"] = (df_feat["ema_20"] / (df_feat["ema_50"] + 1e-12))

# Bollinger Bands
ma20, upper20, lower20, width20, pctb20 = bollinger(close, period=20, num_std=2.0)
df_feat["bb_ma20"] = ma20.fillna(method="bfill")
df_feat["bb_upper20"] = upper20.fillna(method="bfill")
df_feat["bb_lower20"] = lower20.fillna(method="bfill")
df_feat["bb_width20"] = width20.fillna(method="bfill")
df_feat["bb_pctb20"] = pctb20.fillna(method="bfill")

# Target: next close
df_feat["target_next_close"] = df["close"].shift(-1)

# Drop last row with NaN target
df_feat = df_feat.iloc[:-1].copy()

# -----------------------------
# Train/Val/Test Split (time-aware)
# -----------------------------
N = len(df_feat)
test_size = int(N * TEST_RATIO)
val_size = int(N * VAL_RATIO)
train_size = N - val_size - test_size

train_df = df_feat.iloc[:train_size]
val_df   = df_feat.iloc[train_size:train_size+val_size]
test_df  = df_feat.iloc[train_size+val_size:]

feature_cols = [c for c in df_feat.columns if c != "target_next_close"]
target_col = "target_next_close"

X_train = train_df[feature_cols].values
y_train = train_df[target_col].values

X_val = val_df[feature_cols].values
y_val = val_df[target_col].values

X_test = test_df[feature_cols].values
y_test = test_df[target_col].values

# -----------------------------
# Scaling (fit on train only)
# -----------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# -----------------------------
# Sequence Building
# -----------------------------
def build_sequences(X, y, seq_len=SEQ_LEN):
    X_seq, y_seq = [], []
    for i in range(len(X) - seq_len + 1):
        X_seq.append(X[i:i+seq_len])
        # predict next close at the time aligned with the last step in window
        y_seq.append(y[i+seq_len-1])
    return np.array(X_seq, dtype=np.float32), np.array(y_seq, dtype=np.float32)

Xtr_seq, ytr_seq = build_sequences(X_train_scaled, y_train, SEQ_LEN)
Xval_seq, yval_seq = build_sequences(X_val_scaled, y_val, SEQ_LEN)
Xte_seq, yte_seq = build_sequences(X_test_scaled, y_test, SEQ_LEN)

num_features = Xtr_seq.shape[-1]

# -----------------------------
# Positional Encoding
# -----------------------------
def positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    angles = pos * angle_rates
    pe = np.zeros((seq_len, d_model), dtype=np.float32)
    pe[:, 0::2] = np.sin(angles[:, 0::2])
    pe[:, 1::2] = np.cos(angles[:, 1::2])
    return tf.constant(pe)

class AddPositionalEncoding(layers.Layer):
    def __init__(self, seq_len, d_model, **kwargs):
        super().__init__(**kwargs)
        self.seq_len = seq_len
        self.d_model = d_model
        self.pe = positional_encoding(seq_len, d_model)

    def call(self, x):
        # x: (batch, seq_len, d_model)
        return x + self.pe

# -----------------------------
# Transformer Encoder Block
# -----------------------------
def transformer_encoder(x, num_heads=NUM_HEADS, key_dim=KEY_DIM, ff_dim=FF_DIM, dropout=DROPOUT):
    attn_out = layers.MultiHeadAttention(num_heads=num_heads, key_dim=key_dim, dropout=dropout)(x, x)
    x = layers.Add()([x, attn_out])
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    ff = layers.Dense(ff_dim, activation="relu")(x)
    ff = layers.Dropout(dropout)(ff)
    ff = layers.Dense(x.shape[-1], activation=None)(ff)
    x = layers.Add()([x, ff])
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x

# -----------------------------
# Build Model
# -----------------------------
inputs = layers.Input(shape=(SEQ_LEN, num_features))
# Project features to model dimension
x = layers.Dense(KEY_DIM, activation="linear")(inputs)
x = AddPositionalEncoding(seq_len=SEQ_LEN, d_model=KEY_DIM)(x)

for _ in range(NUM_ENCODER_BLOCKS):
    x = transformer_encoder(x, num_heads=NUM_HEADS, key_dim=KEY_DIM, ff_dim=FF_DIM, dropout=DROPOUT)

x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(DROPOUT)(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(DROPOUT)(x)
outputs = layers.Dense(1, activation="linear")(x)

model = models.Model(inputs, outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
              loss="mse",
              metrics=[tf.keras.metrics.MAE, tf.keras.metrics.MAPE])

model.summary()

# -----------------------------
# Training
# -----------------------------
es = callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
rlrp = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-5)
ckpt = callbacks.ModelCheckpoint("xau_transformer_best.h5", monitor="val_loss", save_best_only=True)

history = model.fit(
    Xtr_seq, ytr_seq,
    validation_data=(Xval_seq, yval_seq),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[es, rlrp, ckpt],
    verbose=1
)

# -----------------------------
# Evaluation
# -----------------------------
y_pred = model.predict(Xte_seq, batch_size=BATCH_SIZE).ravel()
mae = mean_absolute_error(yte_seq, y_pred)
mape = mean_absolute_percentage_error(yte_seq, y_pred)
rmse = math.sqrt(np.mean((y_pred - yte_seq)**2))

print(f"Test MAE:  {mae:.4f}")
print(f"Test MAPE: {mape:.4f}")
print(f"Test RMSE: {rmse:.4f}")

# -----------------------------
# Inference: next-step prediction from latest window
# -----------------------------
# Build the last window from test set (or the latest available sequence)
latest_window = X_test_scaled[-SEQ_LEN:]
latest_window = latest_window[np.newaxis, ...]  # shape (1, SEQ_LEN, num_features)
next_close_pred = float(model.predict(latest_window))
print(f"Predicted next close: {next_close_pred:.2f}")


/tmp/ipython-input-2759997566.py:114: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_feat["ret_vol_24"] = df_feat["return"].rolling(window=24, min_periods=24).std().fillna(method="bfill")
/tmp/ipython-input-2759997566.py:115: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_feat["ret_vol_72"] = df_feat["return"].rolling(window=72, min_periods=72).std().fillna(method="bfill")
/tmp/ipython-input-2759997566.py:118: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_feat["atr_14"] = atr(high, low, close, period=14).fillna(method="bfill")
/tmp/ipython-input-2759997566.py:121: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_feat["rsi_14"] = 

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 64, 22)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64, 64)    │      1,472 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_positional_enc… │ (None, 64, 64)    │          0 │ dense[0][0]       │
│ (AddPositionalEnco… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 64, 64)    │     66,368 │ add_positional_e… │
│ (MultiHeadAttentio… │                   │            │ add_positional_e… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 64, 64)    │          0 │ add_positional_e… │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 64, 64)    │        128 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64, 128)   │      8,320 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 64, 128)   │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64, 64)    │      8,256 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 64, 64)    │          0 │ layer_normalizat… │
│                     │                   │            │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 64, 64)    │        128 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 64, 64)    │     66,368 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 64, 64)    │          0 │ layer_normalizat… │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 64, 64)    │        128 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 64, 128)   │      8,320 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 64, 128)   │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 64, 64)    │      8,256 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 64, 64)    │          0 │ layer_normalizat… │
│                     │                   │            │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 64, 64)    │        128 │ add_3[0][0]       │
│ (LayerNormalizatio… │                   │            │                 

 Total params: 172,097 (672.25 KB)

 Trainable params: 172,097 (672.25 KB)

...

158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - loss: 91084472.0000 - mean_absolute_error: 7578.8794 - mean_absolute_percentage_error: 5.8058 - val_loss: 52306532.0000 - val_mean_absolute_error: 3308.0713 - val_mean_absolute_percentage_error: 1.7878 - learning_rate: 0.0010
Epoch 84/100
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 92426272.0000 - mean_absolute_error: 7597.1562 - mean_absolute_percentage_error: 5.8160 - val_loss: 57221572.0000 - val_mean_absolute_error: 4537.0781 - val_mean_absolute_percentage_error: 2.5829 - learning_rate: 0.0010
Epoch 85/100
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 91936496.0000 - mean_absolute_error: 7604.4448 - mean_absolute_percentage_error: 5.8231 - val_loss: 52684304.0000 - val_mean_absolute_error: 3448.8789 - val_mean_absolute_percentage_error: 1.8942 - learning_rate: 0.0010
Epoch 86/100
157/158 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 92089136.0000 - mean_absolute_error: 7601.2114 - mean_absolute_percentage_error: 5.8146

158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 92076304.0000 - mean_absolute_error: 7600.6699 - mean_absolute_percentage_error: 5.8142 - val_loss: 51993084.0000 - val_mean_absolute_error: 3264.1665 - val_mean_absolute_percentage_error: 1.7760 - learning_rate: 0.0010
Epoch 87/100
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 91129216.0000 - mean_absolute_error: 7562.4619 - mean_absolute_percentage_error: 5.7860 - val_loss: 61062292.0000 - val_mean_absolute_error: 4286.9004 - val_mean_absolute_percentage_error: 2.3756 - learning_rate: 0.0010
Epoch 88/100
156/158 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 89450280.0000 - mean_absolute_error: 7483.6768 - mean_absolute_percentage_error: 5.7281

158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - loss: 89459320.0000 - mean_absolute_error: 7484.2246 - mean_absolute_percentage_error: 5.7286 - val_loss: 51825876.0000 - val_mean_absolute_error: 3431.0818 - val_mean_absolute_percentage_error: 1.9140 - learning_rate: 0.0010
Epoch 89/100
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 90280264.0000 - mean_absolute_error: 7547.4116 - mean_absolute_percentage_error: 5.7821 - val_loss: 53632480.0000 - val_mean_absolute_error: 3405.6990 - val_mean_absolute_percentage_error: 1.8645 - learning_rate: 0.0010
Epoch 90/100
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 89831400.0000 - mean_absolute_error: 7520.6455 - mean_absolute_percentage_error: 5.7621 - val_loss: 52319796.0000 - val_mean_absolute_error: 3516.9341 - val_mean_absolute_percentage_error: 1.9232 - learning_rate: 0.0010
Epoch 91/100
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 91970144.0000 - mean_absolute_error: 7589.4321 - mean_absolute_percentage_error: 5.8049 - val_loss: 5

158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 89610656.0000 - mean_absolute_error: 7484.5073 - mean_absolute_percentage_error: 5.7297 - val_loss: 49134716.0000 - val_mean_absolute_error: 3082.3240 - val_mean_absolute_percentage_error: 1.6690 - learning_rate: 5.0000e-04
Epoch 96/100
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 89841656.0000 - mean_absolute_error: 7508.1440 - mean_absolute_percentage_error: 5.7523 - val_loss: 50197948.0000 - val_mean_absolute_error: 3217.9595 - val_mean_absolute_percentage_error: 1.7443 - learning_rate: 5.0000e-04
Epoch 97/100
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 90879880.0000 - mean_absolute_error: 7577.5596 - mean_absolute_percentage_error: 5.8045 - val_loss: 51625140.0000 - val_mean_absolute_error: 3175.6670 - val_mean_absolute_percentage_error: 1.7318 - learning_rate: 5.0000e-04
Epoch 98/100
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 89798728.0000 - mean_absolute_error: 7500.6855 - mean_absolute_percentage_error: 5.7466 -

/tmp/ipython-input-2759997566.py:293: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  next_close_pred = float(model.predict(latest_window))


In [ ]:
import math
import numpy as np
import pandas as pd
from typing import List, Dict, Tuple
import tensorflow
from tensorflow.keras import layers, models, callbacks
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

# -----------------------------
# Config
# -----------------------------
CSV_FILE = "/content/XAUUSDh1.csv"   # path to your CSV
DATE_COL = "Date"
SEQ_LEN = 128              # longer context generally helps
BATCH_SIZE = 256
EPOCHS = 50                # set lower for quick runs; increase after validation
LR = 1e-3
NUM_HEADS = 4
KEY_DIM = 64
FF_DIM = 128
NUM_ENCODER_BLOCKS = 2
DROPOUT = 0.1
EMBED_DIM = 32
TAU = 0.1           # temperature for InfoNCE
ALPHA = 0.2         # weight for contrastive loss vs forecast loss
SEED = 42

# Walk-forward split ratios (fractions of total samples per fold)
INITIAL_TRAIN_RATIO = 0.6
VAL_RATIO = 0.1
TEST_RATIO = 0.1
# The remaining ~0.2 will be covered by subsequent folds via walk-forward

tf.random.set_seed(SEED)
np.random.seed(SEED)

# -----------------------------
# Feature Engineering Utilities
# -----------------------------
def ema(series: pd.Series, span: int) -> pd.Series:
    return series.ewm(span=span, adjust=False).mean()

def rsi(series: pd.Series, period: int = 14) -> pd.Series:
    delta = series.diff()
    gain = np.where(delta > 0, delta, 0.0)
    loss = np.where(delta < 0, -delta, 0.0)
    gain = pd.Series(gain, index=series.index)
    loss = pd.Series(loss, index=series.index)
    avg_gain = gain.ewm(alpha=1/period, min_periods=period, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/period, min_periods=period, adjust=False).mean()
    rs = avg_gain / (avg_loss + 1e-12)
    rsi_val = 100 - (100 / (1 + rs))
    return rsi_val

def true_range(high: pd.Series, low: pd.Series, close: pd.Series) -> pd.Series:
    prev_close = close.shift(1)
    tr1 = (high - low).abs()
    tr2 = (high - prev_close).abs()
    tr3 = (low - prev_close).abs()
    return pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)

def atr(high: pd.Series, low: pd.Series, close: pd.Series, period: int = 14) -> pd.Series:
    tr = true_range(high, low, close)
    return tr.ewm(alpha=1/period, adjust=False).mean()

def macd(close: pd.Series, fast: int = 12, slow: int = 26, signal: int = 9):
    ema_fast = ema(close, fast)
    ema_slow = ema(close, slow)
    macd_line = ema_fast - ema_slow
    signal_line = ema(macd_line, signal)
    hist = macd_line - signal_line
    return macd_line, signal_line, hist

def bollinger(close: pd.Series, period: int = 20, num_std: float = 2.0):
    ma = close.rolling(window=period, min_periods=period).mean()
    sd = close.rolling(window=period, min_periods=period).std()
    upper = ma + num_std * sd
    lower = ma - num_std * sd
    width = upper - lower
    pctb = (close - lower) / (upper - lower + 1e-12)
    return ma, upper, lower, width, pctb

# -----------------------------
# Data Loading and Features
# -----------------------------
def load_and_engineer(csv_file: str, date_col: str) -> pd.DataFrame:
    df = pd.read_csv(csv_file)
    required_cols = {"open", "high", "low", "close", "tick_volume", date_col}
    missing = required_cols - set(df.columns)
    if missing:
        raise ValueError(f"Missing columns in CSV: {missing}")

    df[date_col] = pd.to_datetime(df[date_col])
    df = df.sort_values(date_col).reset_index(drop=True)

    open_ = df["open"].astype(float)
    high = df["high"].astype(float)
    low = df["low"].astype(float)
    close = df["close"].astype(float)
    vol = df["tick_volume"].astype(float)

    feat = pd.DataFrame(index=df.index)
    feat["close"] = close
    feat["ret"] = close.pct_change().fillna(0.0)
    feat["log_ret"] = np.log1p(feat["ret"]).fillna(0.0)
    feat["price_change"] = close.diff().fillna(0.0)
    feat["hl_spread"] = (high - low)
    feat["oc_spread"] = (open_ - close)
    feat["log_vol"] = np.log1p(vol)
    feat["vol_chg"] = vol.diff().fillna(0.0)

    # Volatility
    feat["ret_vol_24"] = feat["ret"].rolling(window=24, min_periods=24).std().fillna(method="bfill")
    feat["ret_vol_72"] = feat["ret"].rolling(window=72, min_periods=72).std().fillna(method="bfill")

    # ATR
    feat["atr_14"] = atr(high, low, close, period=14).fillna(method="bfill")

    # RSI
    feat["rsi_14"] = rsi(close, period=14).fillna(method="bfill")

    # MACD
    macd_line, signal_line, hist = macd(close, fast=12, slow=26, signal=9)
    feat["macd"] = macd_line.fillna(method="bfill")
    feat["macd_signal"] = signal_line.fillna(method="bfill")
    feat["macd_hist"] = hist.fillna(method="bfill")

    # EMAs
    feat["ema_20"] = ema(close, 20).fillna(method="bfill")
    feat["ema_50"] = ema(close, 50).fillna(method="bfill")
    feat["ema_ratio_20_50"] = (feat["ema_20"] / (feat["ema_50"] + 1e-12))

    # Bollinger
    ma20, upper20, lower20, width20, pctb20 = bollinger(close, period=20, num_std=2.0)
    feat["bb_ma20"] = ma20.fillna(method="bfill")
    feat["bb_upper20"] = upper20.fillna(method="bfill")
    feat["bb_lower20"] = lower20.fillna(method="bfill")
    feat["bb_width20"] = width20.fillna(method="bfill")
    feat["bb_pctb20"] = pctb20.fillna(method="bfill")

    # Target: next-step return
    feat["target_next_ret"] = feat["ret"].shift(-1)
    feat["target_next_close"] = df["close"].shift(-1)

    # Drop last row with NaN target
    feat = feat.iloc[:-1].copy()
    feat[date_col] = df[date_col].iloc[:-1].values
    return feat

# -----------------------------
# Sequence/Pair Builders
# -----------------------------
def build_anchor_positive_pairs(X: np.ndarray, y: np.ndarray, seq_len: int) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    Xa, Xp, ya = [], [], []
    # Positive is the adjacent next window (shifted by 1)
    for i in range(len(X) - seq_len - 0):
        j = i + 1
        if j + seq_len - 1 >= len(X):
            break
        Xa.append(X[i:i+seq_len])
        Xp.append(X[j:j+seq_len])
        ya.append(y[i+seq_len-1])  # predict next ret at the end of anchor window
    return np.array(Xa, dtype=np.float32), np.array(Xp, dtype=np.float32), np.array(ya, dtype=np.float32)

def make_dataset(Xa, Xp, ya, batch_size=BATCH_SIZE, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices(((Xa, Xp), ya))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(ya), seed=SEED)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

# -----------------------------
# Positional Encoding
# -----------------------------
def positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    angles = pos * angle_rates
    pe = np.zeros((seq_len, d_model), dtype=np.float32)
    pe[:, 0::2] = np.sin(angles[:, 0::2])
    pe[:, 1::2] = np.cos(angles[:, 1::2])
    return tf.constant(pe)

class AddPositionalEncoding(layers.Layer):
    def __init__(self, seq_len, d_model, **kwargs):
        super().__init__(**kwargs)
        self.pe = positional_encoding(seq_len, d_model)

    def call(self, x):
        return x + self.pe

# -----------------------------
# Transformer Encoder Block
# -----------------------------
class TransformerEncoderBlock(layers.Layer):
    def __init__(self, num_heads, key_dim, ff_dim, dropout=0.1):
        super().__init__()
        self.mha = layers.MultiHeadAttention(num_heads=num_heads, key_dim=key_dim, dropout=dropout)
        self.norm1 = layers.LayerNormalization(epsilon=1e-6)
        self.norm2 = layers.LayerNormalization(epsilon=1e-6)
        self.ffn = models.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dropout(dropout),
            layers.Dense(key_dim)  # project back to key_dim
        ])
        self.dropout = layers.Dropout(dropout)

    def call(self, x, training=False):
        attn_out = self.mha(x, x, training=training)
        x = self.norm1(x + attn_out)
        ffn_out = self.ffn(x, training=training)
        x = self.norm2(x + ffn_out)
        return x

# -----------------------------
# Dual-head Model with custom train_step (InfoNCE + MSE)
# -----------------------------
class DualHeadModel(tf.keras.Model):
    def __init__(self, seq_len, key_dim, embed_dim,
                 num_heads=4, ff_dim=128, num_blocks=2, dropout=0.1, **kwargs):
        super().__init__(**kwargs)
        # Projection + positional encoding
        self.proj = layers.Dense(key_dim)
        self.posenc = AddPositionalEncoding(seq_len, key_dim)
        # Stack of encoder blocks
        self.encoders = [TransformerEncoderBlock(num_heads, key_dim, ff_dim, dropout)
                         for _ in range(num_blocks)]
        # Pooling + dropout
        self.pool = layers.GlobalAveragePooling1D()
        self.dropout = layers.Dropout(dropout)
        # Embedding head
        self.emb_dense = layers.Dense(embed_dim, name="embedding")
        # Regression head
        self.reg_dense1 = layers.Dense(64, activation="relu")
        self.reg_out = layers.Dense(1, name="forecast")
        # Metrics
        self.mae_metric = tf.keras.metrics.MeanAbsoluteError(name="forecast_mae")
        self.mape_metric = tf.keras.metrics.MeanAbsolutePercentageError(name="forecast_mape")

    def encode(self, x, training=False):
        x = self.proj(x)
        x = self.posenc(x)
        for enc in self.encoders:
            x = enc(x, training=training)
        x = self.pool(x)
        x = self.dropout(x, training=training)
        return x

    def call(self, inputs, training=False):
        xa, xp = inputs
        fa = self.encode(xa, training=training)
        fp = self.encode(xp, training=training)
        ea = tf.linalg.l2_normalize(self.emb_dense(fa), axis=1)
        ep = tf.linalg.l2_normalize(self.emb_dense(fp), axis=1)
        # Forecast from anchor path
        r = self.dropout(fa, training=training)
        r = self.reg_dense1(r)
        r = self.dropout(r, training=training)
        yhat = self.reg_out(r)
        return ea, ep, yhat

    def info_nce(self, ea, ep, tau=0.1):
        logits = tf.matmul(ea, ep, transpose_b=True) / tau
        batch_size = tf.shape(logits)[0]
        labels = tf.range(batch_size)
        loss = tf.reduce_mean(
            tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
        )
        return loss

    def train_step(self, data):
        (xa, xp), ya = data
        with tf.GradientTape() as tape:
            ea, ep, yhat = self((xa, xp), training=True)
            forecast_loss = tf.reduce_mean(tf.keras.losses.mse(ya, tf.squeeze(yhat, axis=-1)))
            contrastive_loss = self.info_nce(ea, ep)
            total_loss = forecast_loss + 0.2 * contrastive_loss
        grads = tape.gradient(total_loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        # Metrics
        self.mae_metric.update_state(ya, tf.squeeze(yhat, axis=-1))
        self.mape_metric.update_state(ya, tf.squeeze(yhat, axis=-1))
        return {
            "loss": total_loss,
            "forecast_loss": forecast_loss,
            "contrastive_loss": contrastive_loss,
            "forecast_mae": self.mae_metric.result(),
            "forecast_mape": self.mape_metric.result(),
        }

    def test_step(self, data):
        (xa, xp), ya = data
        ea, ep, yhat = self((xa, xp), training=False)
        forecast_loss = tf.reduce_mean(tf.keras.losses.mse(ya, tf.squeeze(yhat, axis=-1)))
        contrastive_loss = self.info_nce(ea, ep)
        total_loss = forecast_loss + 0.2 * contrastive_loss
        self.mae_metric.update_state(ya, tf.squeeze(yhat, axis=-1))
        self.mape_metric.update_state(ya, tf.squeeze(yhat, axis=-1))
        return {
            "loss": total_loss,
            "forecast_loss": forecast_loss,
            "contrastive_loss": contrastive_loss,
            "forecast_mae": self.mae_metric.result(),
            "forecast_mape": self.mape_metric.result(),
        }

# -----------------------------
# Walk-forward Split Generator
# -----------------------------
def generate_walk_forward_slices(N: int,
                                 initial_train_ratio: float,
                                 val_ratio: float,
                                 test_ratio: float) -> List[Tuple[int, int, int, int, int, int]]:
    """
    Returns list of (train_start, train_end, val_start, val_end, test_start, test_end) indices.
    Expanding train window; fixed val/test windows; step forward by test window.
    """
    initial_train = int(N * initial_train_ratio)
    val_len = int(N * val_ratio)
    test_len = int(N * test_ratio)
    slices = []

    train_start = 0
    train_end = initial_train
    val_start = train_end
    val_end = val_start + val_len
    test_start = val_end
    test_end = test_start + test_len

    while test_end <= N:
        slices.append((train_start, train_end, val_start, val_end, test_start, test_end))
        # advance: expand train to include previous val+test, then new val/test slices
        train_end = test_end
        val_start = train_end
        val_end = val_start + val_len
        test_start = val_end
        test_end = test_start + test_len

    return slices

# -----------------------------
# Fold Runner
# -----------------------------
def run_fold(feat_df: pd.DataFrame,
             date_col: str,
             indices: Tuple[int, int, int, int, int, int]) -> Dict[str, float]:
    ts, te, vs, ve, ss, se = indices

    train_df = feat_df.iloc[ts:te]
    val_df   = feat_df.iloc[vs:ve]
    test_df  = feat_df.iloc[ss:se]

    feature_cols = [c for c in feat_df.columns if c not in ["target_next_ret", "target_next_close", date_col]]
    target_col = "target_next_ret"

    # Fit scalers on train
    X_scaler = StandardScaler()
    y_scaler = StandardScaler()

    X_train = train_df[feature_cols].values
    y_train = train_df[target_col].values.reshape(-1, 1)

    X_val = val_df[feature_cols].values
    y_val = val_df[target_col].values.reshape(-1, 1)

    X_test = test_df[feature_cols].values
    y_test = test_df[target_col].values.reshape(-1, 1)

    X_train_scaled = X_scaler.fit_transform(X_train)
    X_val_scaled   = X_scaler.transform(X_val)
    X_test_scaled  = X_scaler.transform(X_test)

    y_train_scaled = y_scaler.fit_transform(y_train).ravel()
    y_val_scaled   = y_scaler.transform(y_val).ravel()
    y_test_scaled  = y_scaler.transform(y_test).ravel()

    Xa_tr, Xp_tr, ya_tr = build_anchor_positive_pairs(X_train_scaled, y_train_scaled, SEQ_LEN)
    Xa_val, Xp_val, ya_val = build_anchor_positive_pairs(X_val_scaled, y_val_scaled, SEQ_LEN)
    Xa_te, Xp_te, ya_te = build_anchor_positive_pairs(X_test_scaled, y_test_scaled, SEQ_LEN)

    # If sequences are too few, skip this fold
    if len(ya_tr) == 0 or len(ya_val) == 0 or len(ya_te) == 0:
        return {"skip": 1}

    train_ds = make_dataset(Xa_tr, Xp_tr, ya_tr, BATCH_SIZE, shuffle=True)
    val_ds   = make_dataset(Xa_val, Xp_val, ya_val, BATCH_SIZE, shuffle=False)
    test_ds  = make_dataset(Xa_te, Xp_te, ya_te, BATCH_SIZE, shuffle=False)

    # Build and compile model
    model = DualHeadModel(seq_len=SEQ_LEN, key_dim=KEY_DIM, embed_dim=EMBED_DIM)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR))

    # Callbacks
    es = callbacks.EarlyStopping(monitor="val_loss", patience=7, restore_best_weights=True)
    rlrp = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=4, min_lr=1e-5)
    ckpt = callbacks.ModelCheckpoint(f"xau_dualhead_fold_{ts}_{se}.h5", monitor="val_loss", save_best_only=True)

    # Train
    history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS,
                        callbacks=[es, rlrp, ckpt], verbose=1)

    # Evaluate on test
    eval_out = model.evaluate(test_ds, verbose=0)

    # Collect regression predictions for metrics on price
    y_true_ret = []
    y_pred_ret = []
    last_close = []  # for reconstructing predicted next close

    # Align closes for windows: at each anchor window end, use the actual last close to reconstruct next close
    closes = test_df["close"].values
    # The close aligned with the end of each anchor window starts at index SEQ_LEN-1 within test_df
    anchor_end_close = closes[SEQ_LEN-1:len(test_df)-0]

    idx = 0
    for (xa_batch, xp_batch), ya_batch in test_ds:
        _, _, yhat_batch = model((xa_batch, xp_batch), training=False)
        y_true_ret.append(ya_batch.numpy())
        y_pred_ret.append(tf.squeeze(yhat_batch, axis=-1).numpy())
        # prepare last_close slice matching batch length
        batch_len = ya_batch.shape[0]
        last_close.extend(anchor_end_close[idx:idx+batch_len])
        idx += batch_len

    y_true_ret = np.concatenate(y_true_ret)
    y_pred_ret_scaled = np.concatenate(y_pred_ret)

    # Invert target scaling
    y_true_ret = y_true_ret.reshape(-1, 1)
    y_pred_ret_scaled = y_pred_ret_scaled.reshape(-1, 1)
    y_true_ret = y_scaler.inverse_transform(y_true_ret).ravel()
    y_pred_ret = y_scaler.inverse_transform(y_pred_ret_scaled).ravel()

    # Reconstruct next close from returns: next_close = last_close * (1 + ret)
    last_close = np.array(last_close)
    y_pred_close = last_close * (1.0 + y_pred_ret)
    # Ground-truth next close aligned to anchor window ends
    target_next_close = test_df["target_next_close"].values[SEQ_LEN-1:SEQ_LEN-1+len(y_pred_close)]

    # Metrics on price
    mae = mean_absolute_error(target_next_close, y_pred_close)
    mape = mean_absolute_percentage_error(target_next_close, y_pred_close)
    rmse = math.sqrt(np.mean((y_pred_close - target_next_close)**2))

    # Baselines:
    # - Persistence baseline: next close = last close (i.e., ret baseline = 0)
    baseline_close = last_close
    baseline_mae = mean_absolute_error(target_next_close, baseline_close)
    baseline_mape = mean_absolute_percentage_error(target_next_close, baseline_close)
    baseline_rmse = math.sqrt(np.mean((baseline_close - target_next_close)**2))

    return {
        "skip": 0,
        "fold_train_start": ts,
        "fold_test_end": se,
        "val_loss": float(eval_out["loss"]) if isinstance(eval_out, dict) and "loss" in eval_out else float(eval_out[0]),
        "mae": float(mae),
        "mape": float(mape),
        "rmse": float(rmse),
        "baseline_mae": float(baseline_mae),
        "baseline_mape": float(baseline_mape),
        "baseline_rmse": float(baseline_rmse),
    }

# -----------------------------
# Main: Run Walk-forward
# -----------------------------
def main():
    feat_df = load_and_engineer(CSV_FILE, DATE_COL)
    N = len(feat_df)
    slices = generate_walk_forward_slices(
        N,
        INITIAL_TRAIN_RATIO,
        VAL_RATIO,
        TEST_RATIO
    )

    results = []
    for idx, s in enumerate(slices, 1):
        print(f"\n=== Fold {idx}/{len(slices)} ===")
        ts, te, vs, ve, ss, se = s
        print(f"Train: {ts}-{te}, Val: {vs}-{ve}, Test: {ss}-{se}")
        res = run_fold(feat_df, DATE_COL, s)
        if res.get("skip", 0) == 1:
            print("Skipping fold (insufficient sequence length).")
            continue
        print(f"Fold {idx} Metrics: "
              f"MAE={res['mae']:.2f}, MAPE={res['mape']:.4f}, RMSE={res['rmse']:.2f} | "
              f"Baseline MAE={res['baseline_mae']:.2f}, Baseline MAPE={res['baseline_mape']:.4f}, "
              f"Baseline RMSE={res['baseline_rmse']:.2f}")
        results.append(res)

    if results:
        # Aggregate
        avg_mae = np.mean([r["mae"] for r in results])
        avg_mape = np.mean([r["mape"] for r in results])
        avg_rmse = np.mean([r["rmse"] for r in results])

        avg_b_mae = np.mean([r["baseline_mae"] for r in results])
        avg_b_mape = np.mean([r["baseline_mape"] for r in results])
        avg_b_rmse = np.mean([r["baseline_rmse"] for r in results])

        print("\n=== Aggregate Walk-forward Results ===")
        print(f"Model:   MAE={avg_mae:.2f}, MAPE={avg_mape:.4f}, RMSE={avg_rmse:.2f}")
        print(f"Baseline MAE={avg_b_mae:.2f}, MAPE={avg_b_mape:.4f}, RMSE={avg_b_rmse:.2f}")

    # Optional: final inference on the latest available window of the last fold
    # Build a latest window from the last test slice
    if slices:
        _, _, _, _, ss, se = slices[-1]
        last_test = feat_df.iloc[ss:se]
        feature_cols = [c for c in feat_df.columns if c not in ["target_next_ret", "target_next_close", DATE_COL]]

        X_scaler = StandardScaler()
        y_scaler = StandardScaler()

        # Fit on train+val of the last fold for inference
        ts, te, vs, ve, *_ = slices[-1]
        fit_df = feat_df.iloc[ts:ve]
        X_fit = fit_df[feature_cols].values
        y_fit = fit_df["target_next_ret"].values.reshape(-1, 1)
        X_scaler.fit(X_fit)
        y_scaler.fit(y_fit)

        X_test_scaled = X_scaler.transform(last_test[feature_cols].values)
        closes = last_test["close"].values

        # Create anchor/positive for latest window
        if len(X_test_scaled) >= SEQ_LEN + 1:
            anchor = X_test_scaled[-SEQ_LEN:]
            positive = X_test_scaled[-SEQ_LEN+1:]
            anchor = anchor[np.newaxis, ...]
            positive = positive[np.newaxis, ...]

            # Build and load a fresh model for inference
            model = DualHeadModel(seq_len=SEQ_LEN, key_dim=KEY_DIM, embed_dim=EMBED_DIM)
            # Not loading weights here (no file path management in this template); train your final model then load for production

            # Forward pass (untrained in this snippet)
            ea, ep, yhat = model((anchor, positive), training=False)
            pred_ret_scaled = float(tf.squeeze(yhat, axis=-1).numpy().ravel()[0])
            pred_ret = float(y_scaler.inverse_transform([[pred_ret_scaled]]).ravel()[0])

            latest_close = float(closes[-1])
            next_close_pred = latest_close * (1.0 + pred_ret)
            print(f"\nLatest inference (untrained): predicted next close = {next_close_pred:.2f}")

if __name__ == "__main__":
    main()


/tmp/ipython-input-824757826.py:115: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feat["ret_vol_24"] = feat["ret"].rolling(window=24, min_periods=24).std().fillna(method="bfill")
/tmp/ipython-input-824757826.py:116: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feat["ret_vol_72"] = feat["ret"].rolling(window=72, min_periods=72).std().fillna(method="bfill")
/tmp/ipython-input-824757826.py:119: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feat["atr_14"] = atr(high, low, close, period=14).fillna(method="bfill")
/tmp/ipython-input-824757826.py:122: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feat["rsi_14"] = rsi(close, period=14).fillna


=== Fold 1/2 ===
Train: 0-34559, Val: 34559-40318, Test: 40318-46077
Epoch 1/50
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step - contrastive_loss: 1.8609 - forecast_loss: 1.0414 - forecast_mae: 0.6679 - forecast_mape: 263.9066 - loss: 1.4136

135/135 ━━━━━━━━━━━━━━━━━━━━ 47s 198ms/step - contrastive_loss: 1.8528 - forecast_loss: 1.0405 - forecast_mae: 0.6678 - forecast_mape: 263.6139 - loss: 1.4111 - val_contrastive_loss: 4.6416 - val_forecast_loss: 0.4689 - val_forecast_mae: 0.4411 - val_forecast_mape: 186.8220 - val_loss: 1.3972 - learning_rate: 0.0010
Epoch 2/50
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - contrastive_loss: 0.8092 - forecast_loss: 1.0105 - forecast_mae: 0.6345 - forecast_mape: 179.2558 - loss: 1.1723

135/135 ━━━━━━━━━━━━━━━━━━━━ 12s 88ms/step - contrastive_loss: 0.8055 - forecast_loss: 1.0089 - forecast_mae: 0.6345 - forecast_mape: 179.2266 - loss: 1.1700 - val_contrastive_loss: 3.4530 - val_forecast_loss: 0.4629 - val_forecast_mae: 0.4327 - val_forecast_mape: 146.1655 - val_loss: 1.1535 - learning_rate: 0.0010
Epoch 3/50
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - contrastive_loss: 0.4889 - forecast_loss: 1.0037 - forecast_mae: 0.6300 - forecast_mape: 160.4449 - loss: 1.1014

135/135 ━━━━━━━━━━━━━━━━━━━━ 12s 90ms/step - contrastive_loss: 0.4868 - forecast_loss: 1.0000 - forecast_mae: 0.6300 - forecast_mape: 160.4582 - loss: 1.0973 - val_contrastive_loss: 3.3255 - val_forecast_loss: 0.4612 - val_forecast_mae: 0.4331 - val_forecast_mape: 134.6468 - val_loss: 1.1263 - learning_rate: 0.0010
Epoch 4/50
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - contrastive_loss: 0.3880 - forecast_loss: 1.0015 - forecast_mae: 0.6225 - forecast_mape: 158.0280 - loss: 1.0790

135/135 ━━━━━━━━━━━━━━━━━━━━ 12s 90ms/step - contrastive_loss: 0.3868 - forecast_loss: 1.0000 - forecast_mae: 0.6225 - forecast_mape: 158.0040 - loss: 1.0774 - val_contrastive_loss: 2.9308 - val_forecast_loss: 0.4634 - val_forecast_mae: 0.4316 - val_forecast_mape: 123.5976 - val_loss: 1.0495 - learning_rate: 0.0010
Epoch 5/50
 89/135 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - contrastive_loss: 0.3548 - forecast_loss: 1.0045 - forecast_mae: 0.6209 - forecast_mape: 146.0966 - loss: 1.0755

In [4]:
df

,Date,open,high,low,close,tick_volume
0,2012-05-17 08:00:00,154759.0,155384.0,154740.0,155295.0,4418
1,2012-05-17 09:00:00,155297.0,155297.0,154837.0,154966.0,5512
2,2012-05-17 10:00:00,154975.0,155055.0,154563.0,154801.0,6503
3,2012-05-17 11:00:00,154798.0,155050.0,154654.0,154952.0,5950
4,2012-05-17 12:00:00,154978.0,154978.0,154619.0,154805.0,6485
...,...,...,...,...,...,...
57595,2022-03-04 19:00:00,196265.0,196536.0,196003.0,196212.0,6657
57596,2022-03-04 20:00:00,196212.0,196647.0,196080.0,196512.0,5153
57597,2022-03-04 21:00:00,196524.0,196996.0,196454.0,196706.0,5393
57598,2022-03-04 22:00:00,196699.0,197019.0,196519.0,196733.0,6665
